# 1. Load the dataset and pre-process the images 

In [74]:
#Import the required libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
import shutil
from sklearn.model_selection import train_test_split

In [75]:
# Set the path to the dataset
dataset_path = "C:/Users/Dell/Desktop/New_Tomato/PlantVillage"

In [66]:
# Define the image size and batch size
image_size = (256, 256)
batch_size = 64

In [67]:
# Create data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=(0.8, 1.2),
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

In [68]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [69]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [73]:
training_set = train_datagen.flow_from_directory('train_data',
target_size=(224,224),
batch_size= 64,
class_mode='categorical')

Found 1925 images belonging to 1 classes.


In [71]:
validation_set = test_datagen.flow_from_directory('val_data',
target_size=(224, 224),
batch_size= 64,
class_mode='categorical')

Found 642 images belonging to 1 classes.


In [72]:
test_set = test_datagen.flow_from_directory('test_data',
target_size=(224, 224),
batch_size= 64,
class_mode='categorical')

Found 641 images belonging to 1 classes.


# 2. Load the pre-trained MobileNetV2 model

In [39]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights="imagenet",
                         input_shape=(224, 224, 3), 
                         include_top=False)

In [40]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [41]:
# Add custom layers
num_classes = 6
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# 3.Train the model with fine-tuning

In [42]:
# Fine-tune the model
base_model.trainable = True
fine_tune_at = 100

In [43]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [44]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

In [45]:
history = model.fit(training_set,
                    validation_data= validation_set,
                    epochs=20,
                    steps_per_epoch=len(training_set),
                    validation_steps=len(validation_set))

Epoch 1/20
860/860 [==============================] - 989s 1s/step - loss: 0.4371 - accuracy: 0.8421 - val_loss: 3.2242 - val_accuracy: 0.4328
Epoch 2/20
860/860 [==============================] - 910s 1s/step - loss: 0.1443 - accuracy: 0.9529 - val_loss: 1.4615 - val_accuracy: 0.6866
Epoch 3/20
860/860 [==============================] - 912s 1s/step - loss: 0.0935 - accuracy: 0.9697 - val_loss: 1.2331 - val_accuracy: 0.7164
Epoch 4/20
860/860 [==============================] - 959s 1s/step - loss: 0.0753 - accuracy: 0.9769 - val_loss: 1.1136 - val_accuracy: 0.7164
Epoch 5/20
860/860 [==============================] - 948s 1s/step - loss: 0.0600 - accuracy: 0.9814 - val_loss: 1.1569 - val_accuracy: 0.6866
Epoch 6/20
860/860 [==============================] - 926s 1s/step - loss: 0.0582 - accuracy: 0.9820 - val_loss: 1.6779 - val_accuracy: 0.7313
Epoch 7/20
860/860 [==============================] - 986s 1s/step - loss: 0.0468 - accuracy: 0.9855 - val_loss: 1.2463 - val_accuracy: 0.7164

UnknownError: Graph execution error:

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Dell/Desktop/Tomato Leaf disease detection/dataset2/valid\\Tomato Early blight\\TomatoEarlyBlight(10).JPG'
Traceback (most recent call last):

  File "C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1039, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py", line 901, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\Del

# 4. Evaluate the model

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(history.history['val_accuracy'],linewidth = 4)
plt.plot(history.history['accuracy'],linewidth = 4)

plt.title("Training and Validation Accuracy",fontsize=12)
plt.ylabel("Accuracy",fontsize=12)
plt.xlabel("Number of epochs",fontsize=12)
plt.legend(['val','train'],loc = 'lower right')
plt.grid()

plt.show()

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(history.history['loss'],linewidth = 4)
plt.plot(history.history['val_loss'],linewidth = 4)

plt.title("Training and Validation Loss",fontsize=12)
plt.ylabel("Loss",fontsize=12)
plt.xlabel("Number of epochs",fontsize=12)
plt.legend(['train','val'],loc = 'upper right')
plt.grid()
plt.show()

In [36]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model.evaluate(training_set)
print(f"Validation loss: {val_loss:.4f}, Validation accuracy: {val_acc:.4f}")

188/188 [==============================] - 157s 831ms/step - loss: 0.3362 - accuracy: 0.8993
Validation loss: 0.3362, Validation accuracy: 0.8993


In [22]:
import cv2
img_test = cv2.imread('dataset/val/Tomato___Tomato_Yellow_Leaf_Curl_Virus/1bef6514-c4be-4417-a390-3c506d4f1404___UF.GRC_YLCV_Lab 02020.JPG')
img_resize = cv2.resize(img_test,(256,256))
#plt.imshow(mpimg.imread('dataset/val/Tomato___Tomato_Yellow_Leaf_Curl_Virus/1bef6514-c4be-4417-a390-3c506d4f1404___UF.GRC_YLCV_Lab 02020.JPG'))
img_scaled = img_resize/255

#BGR to RGB
img_rgb = np.flip(img_scaled, axis=-1)

plt.imshow(img_rgb)
img_reshaped = np.reshape(img_rgb,[1,256,256,3])
#img_array = img_to_array(img_rgb) #It is better to use these two lines instead of img_reshaped = np.reshape(img_rgb,[1,256,256,3]). Both give same result but these do not depend on the scenario.
#img_reshaped = img_array.reshape((1,) + img_array.shape)

input_pred = model.predict(img_reshaped)
print(input_pred)
input_label = np.argmax(input_pred)
print(input_label)


if input_label == 0:
    print("Early_blight")
elif input_label == 1:
    print("Healthy")
elif input_label == 2:
    print("Late_blight")
elif input_label == 3:
    print("Septoria_leaf_spot")
elif input_label == 4:
    print("Target_Spot")
elif input_label == 5:
    print("Tomato_Yellow_Leaf_Curl_Virus")

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


# 5. Save the model

In [37]:
# Define the version string for the saved model
version = "1.0.0"

In [38]:
# Save the model with version details
model.save(f"tomato_disease_classifier_v{version.replace('.', '_')}.h5")

In [41]:
# Load the saved Keras model
model = tf.keras.models.load_model("tomato_disease_classifier_v1_0_0.h5")

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open("tomato_disease_classifier_v1_0_0.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\Dell\AppData\Local\Temp\tmpl_iba472\assets


INFO:tensorflow:Assets written to: C:\Users\Dell\AppData\Local\Temp\tmpl_iba472\assets


In [20]:
# Load the saved model
model = load_model("tomato_disease_classifier_v_1_0_0.h5")

NameError: name 'load_model' is not defined

In [ ]:
# Evaluate the model on the test images
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

In [ ]:
# Make predictions on the test images
test_predictions = model.predict(test_generator, verbose=1)